In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import haversine as hs
from haversine import Unit

In [13]:
# read rental data
path = os.getcwd().replace("notebooks","") + "data/curated/"
external = pd.read_csv(path+'external.csv', low_memory = False)
rental = pd.read_csv(path+'rental_median.csv', low_memory = False)
GNR = pd.read_csv(path+'GNR_suburb.csv', low_memory = False)
property_df = pd.read_csv(path+'cleaned_rent.csv', low_memory = False)
GNR = GNR.drop(columns=["PLACE_ID","geometry"])
GNR["coordinates"] = list(zip(GNR.LATITUDE, GNR.LONGITUDE))
GNR = GNR.drop_duplicates(subset='coordinates', keep="first")

In [14]:
GNR

,PLACE_NAME,FEATURE,LONGITUDE,LATITUDE,coordinates
0,BALLARAT FIRE STATION,FIRE STATION,143.8686,-37.563,"(-37.563, 143.8686)"
1,BUNINYONG FIRE STATION,FIRE STATION,143.8851,-37.650,"(-37.65, 143.8851)"
2,INVERMAY FIRE STATION,FIRE STATION,143.8724,-37.521,"(-37.521, 143.8724)"
3,MINERS REST FIRE STATION,FIRE STATION,143.7989,-37.479,"(-37.479, 143.7989)"
4,WENDOUREE FIRE STATION,FIRE STATION,143.8197,-37.535,"(-37.535, 143.8197)"
...,...,...,...,...,...
13955,THOMSONS BEACH,BEACH,145.6673,-35.912,"(-35.912, 145.6673)"
13956,Sandy Waterhole Beach,BEACH,145.4372,-38.540,"(-38.54, 145.4372)"
13957,No 2 Quay,HARBOUR,141.6193,-38.353,"(-38.353, 141.6193)"
13958,CORIO QUAY,HARBOUR,144.3644,-38.106,"(-38.106, 144.3644)"


In [ ]:
df = GNR[GNR["FEATURE"] == "GENERAL HOSPITAL"]
df.sort_values(by='LATITUDE', ascending=False)
df

In [15]:
# combine longtitude and latitude to coordnates and use first 10 property data as sample
property_df["coordinates"] = list(zip(property_df.latitude, property_df.longitude))
property_df = property_df.iloc[:10]
property_df

,address,rent,features,type,furnitured,pool,gym,latitude,longitude,postcode,num_bed,num_bath,num_park,rent_weekly,coordinates
0,1414/218-228 A'Beckett Street Melbourne VIC 3000,$400 Per Week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.810283,144.956669,3000,1,1,0,400,"(-37.8102832, 144.9566691)"
1,11a/131 Lonsdale Sreet Melbourne VIC 3000,$350 per week,1 Bed1 Bath− Parking,Studio,Yes,No,No,-37.810779,144.968551,3000,1,1,0,350,"(-37.810779, 144.9685513)"
2,911/408 Lonsdale Street Melbourne VIC 3000,$330 per week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.812598,144.960401,3000,1,1,0,330,"(-37.8125979, 144.9604012)"
3,918/422 Collins St Melbourne VIC 3000,$600 Per week fully furnished,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.817097,144.960149,3000,2,1,0,600,"(-37.8170971, 144.9601487)"
4,602/118 Franklin Street Melbourne VIC 3000,$330,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.808205,144.958903,3000,1,1,0,330,"(-37.8082052, 144.9589035)"
5,1112/333-351 Exhibition Street Melbourne VIC 3000,$600 per week,2 Beds2 Baths− Parking,Apartment / Unit / Flat,Yes,No,No,-37.807896,144.968287,3000,2,2,0,600,"(-37.80789559999999, 144.9682873)"
6,3002/288 Spencer St Melbourne VIC 3000,$510,2 Beds1 Bath1 Parking,Apartment / Unit / Flat,No,Yes,Yes,-37.813775,144.952095,3000,2,1,1,510,"(-37.813775, 144.9520948)"
7,4/180 Little Collins Street Melbourne VIC 3000,$500 per week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,Yes,No,No,-37.813860,144.967907,3000,1,1,0,500,"(-37.8138601, 144.9679067)"
8,1605/565 Flinders Street Melbourne VIC 3000,$500 per week,2 Beds2 Baths1 Parking,Apartment / Unit / Flat,No,No,No,-37.821059,144.955907,3000,2,2,1,500,"(-37.8210586, 144.9559072)"
9,612/408 Lonsdale Street Melbourne VIC 3000,$350,1 Bed1 Bath− Parking,Apartment / Unit / Flat,Yes,No,No,-37.812598,144.960401,3000,1,1,0,350,"(-37.8125979, 144.9604012)"


In [40]:
# generate a dict for each property data that contains distance and coordination of cloest three features
def generate_dict(features):
    dist_lis = {}
    # initialize feature
    for feature in features:
        dist_lis[feature] = []
    return dist_lis

# comparing the distances and record three cloest points
def cloest_point(point,dist_dict,dist,loc):
    # if there is no point, then just append the point into dict
    if len(dist_dict[point["FEATURE"]]) < 3:
        dist_dict[point["FEATURE"]].append((dist,loc,point["PLACE_NAME"]))
    # if there already has three points, then compares distance
    else:
        # if the current distance smaller than records' distance
        if dist < dist_dict[point["FEATURE"]][0][0]:
            # delete the record point
            dist_dict[point["FEATURE"]].pop(0)
            # append new cloest point
            dist_dict[point["FEATURE"]].append((dist,loc,point["PLACE_NAME"]))
        elif dist < dist_dict[point["FEATURE"]][1][0]:
            # delete the record point
            dist_dict[point["FEATURE"]].pop(1)
            # append new cloest point
            dist_dict[point["FEATURE"]].append((dist,loc,point["PLACE_NAME"]))
        elif dist < dist_dict[point["FEATURE"]][2][0]:
            # delete the record point
            dist_dict[point["FEATURE"]].pop(2)
            # append new cloest point
            dist_dict[point["FEATURE"]].append((dist,loc,point["PLACE_NAME"]))
    return dist_dict
            
# calculate the cloest three point of interest for each property data
def distance(loc1):
    # read point of interest data
    GNR = pd.read_csv(path+'GNR_suburb.csv', low_memory = False)
    # extact all features
    Point_of_Interest = list(GNR["FEATURE"].unique())
    # initaliza the dict for record the cloest three point of interest
    dist_lis = generate_dict(Point_of_Interest)
    # find cloest three points
    for feature in Point_of_Interest:
        df = GNR[GNR["FEATURE"] == feature]  # select feature data
        # calculate  distances for all feature points
        for i in range(df.shape[0]):
            # feature points
            loc2 = (df.iloc[i]["LATITUDE"],df.iloc[i]["LONGITUDE"])
            # calculate distance between property and feature
            dist = hs.haversine(loc1,loc2,unit=Unit.METERS)
            # check the cloest
            dist_lis = cloest_point(df.iloc[i],dist_lis,dist,loc2)
    return dist_lis

In [41]:
property_df["Distance"]= property_df["coordinates"].apply(distance)  # calculate distance for each property row

In [73]:
# extract hospital data
def get_hospital_dist(dist_dict):
    return dist_dict["GENERAL HOSPITAL"]
property_df["Hospital Distance"]= property_df["Distance"].apply(get_hospital_dist)
property_df.head(5)

,address,rent,features,type,furnitured,pool,gym,latitude,longitude,postcode,num_bed,num_bath,num_park,rent_weekly,coordinates,Distance,Hospital Distance
0,1414/218-228 A'Beckett Street Melbourne VIC 3000,$400 Per Week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.810283,144.956669,3000,1,1,0,400,"(-37.8102832, 144.9566691)","{'FIRE STATION': [(43525.65645407155, (-37.963...","[(97.08047913792757, (-37.811, 144.9573), PETE..."
1,11a/131 Lonsdale Sreet Melbourne VIC 3000,$350 per week,1 Bed1 Bath− Parking,Studio,Yes,No,No,-37.810779,144.968551,3000,1,1,0,350,"(-37.810779, 144.9685513)","{'FIRE STATION': [(44466.49523788073, (-37.963...","[(1488.0248111071182, (-37.807, 144.9848), AFT..."
2,911/408 Lonsdale Street Melbourne VIC 3000,$330 per week,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.812598,144.960401,3000,1,1,0,330,"(-37.8125979, 144.9604012)","{'FIRE STATION': [(43727.74103850396, (-37.963...","[(325.2521090634637, (-37.811, 144.9573), PETE..."
3,918/422 Collins St Melbourne VIC 3000,$600 Per week fully furnished,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.817097,144.960149,3000,2,1,0,600,"(-37.8170971, 144.9601487)","{'FIRE STATION': [(43517.15113459328, (-37.963...","[(1277.168753849104, (-37.825, 144.9707), PRIN..."
4,602/118 Franklin Street Melbourne VIC 3000,$330,1 Bed1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.808205,144.958903,3000,1,1,0,330,"(-37.8082052, 144.9589035)","{'FIRE STATION': [(43797.08541655511, (-37.963...","[(341.2042915199184, (-37.811, 144.9573), PETE..."


In [75]:
# select columns we need
property_df = property_df[['address','latitude', 'longitude','coordinates','Hospital Distance']]
property_df.head(5)

,address,latitude,longitude,coordinates,Hospital Distance
0,1414/218-228 A'Beckett Street Melbourne VIC 3000,-37.810283,144.956669,"(-37.8102832, 144.9566691)","[(97.08047913792757, (-37.811, 144.9573), PETE..."
1,11a/131 Lonsdale Sreet Melbourne VIC 3000,-37.810779,144.968551,"(-37.810779, 144.9685513)","[(1488.0248111071182, (-37.807, 144.9848), AFT..."
2,911/408 Lonsdale Street Melbourne VIC 3000,-37.812598,144.960401,"(-37.8125979, 144.9604012)","[(325.2521090634637, (-37.811, 144.9573), PETE..."
3,918/422 Collins St Melbourne VIC 3000,-37.817097,144.960149,"(-37.8170971, 144.9601487)","[(1277.168753849104, (-37.825, 144.9707), PRIN..."
4,602/118 Franklin Street Melbourne VIC 3000,-37.808205,144.958903,"(-37.8082052, 144.9589035)","[(341.2042915199184, (-37.811, 144.9573), PETE..."
